In [1]:
pip install openai

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 10.3 MB/s  0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 22.5 MB/s  0:00:00

   ----- ---------------------------------- 1/7 [pydantic-core]
   ---------------------------- ----------- 5/7 [pydantic]
   ---------------------------- ----------- 5/7 [pydantic]
   ---------------------------- ----------- 5/7 [pydantic]
   ---------------------------- ----------- 5/7 [pydantic]
   ---------------------------- ----------- 5/7 [pydantic]
   ---------------------------- ----------- 5/7 [pydantic]
   ---------------------------- ----------- 5/7 [pydantic]
   ---------------------------------- ----- 6/7 [openai]
   ---------------------------------- ----- 6/7 [openai]
   ---------------------------------- ----- 6/7 [openai]
   ---------------------------------- ----- 6/7 [openai]
   -----

In [5]:
! pip install azure-ai-documentintelligence
! pip install beautifulsoup4

In [ ]:
#Get document intelligence resource
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import os

load_dotenv()

doc_intel_endpoint = os.getenv("DOC_INTEL_ENDPOINT")
doc_intel_key = os.getenv("DOC_INTEL_KEY")
doc_intel_client = DocumentIntelligenceClient(endpoint=doc_intel_endpoint, credential=AzureKeyCredential(doc_intel_key))

In [19]:
from bs4 import BeautifulSoup
#Generate markdowns
input_dir ="./project-assurance-data"
output_dir = './markdown_files'
os.makedirs(output_dir,exist_ok=True)

for filename in os.listdir(input_dir):
    with open(input_dir+"/"+filename, "rb") as file:
        html_content = file.read()
    try:
        # Convert to markdown with Document Intelligence
        poller = doc_intel_client.begin_analyze_document("prebuilt-layout",
                                                        body=html_content,
                                                        content_type="application/octet-stream")
        result = poller.result()
        markdown = result['content']

    except Exception as ex:
        print(ex)
        # If Document Intelligence fails to convert HTML to markdown, use Beautiful soap
        soup = BeautifulSoup(html_content, "html.parser")
        markdown = soup.get_text()
        
    new_filename = filename[:-4]+"txt"
    with open(output_dir+"/"+new_filename, "w", encoding="utf-8") as file:
        file.write(markdown)


(InvalidRequest) Invalid request.
Code: InvalidRequest
Message: Invalid request.
Exception Details:	(UnsupportedContent) Content is not supported: The input content is corrupted or format is invalid.
	Code: UnsupportedContent
	Message: Content is not supported: The input content is corrupted or format is invalid.
	Target: 0
(InvalidRequest) Invalid request.
Code: InvalidRequest
Message: Invalid request.
Exception Details:	(UnsupportedContent) Content is not supported: The input content is corrupted or format is invalid.
	Code: UnsupportedContent
	Message: Content is not supported: The input content is corrupted or format is invalid.
	Target: 0
(InvalidRequest) Invalid request.
Code: InvalidRequest
Message: Invalid request.
Exception Details:	(UnsupportedContent) Content is not supported: The input content is corrupted or format is invalid.
	Code: UnsupportedContent
	Message: Content is not supported: The input content is corrupted or format is invalid.
	Target: 0
(InvalidRequest) Inval

In [20]:
! pip install --upgrade --quiet langchain-text-splitters tiktoken

In [40]:
from langchain_text_splitters import TokenTextSplitter
#Chunking
chunk_dir = "./chunks"
MAX_CHUNK_TOKEN_SIZE = 512 # Max number of tokens for chunking
CHUNK_OVERLAPPING=0.25 # 25% of overlapping between chunks

os.makedirs(chunk_dir, exist_ok=True)
for filename in os.listdir(output_dir):
    with open(output_dir+"/"+filename, "r", encoding="utf-8") as file:
        text = file.read()
    
    text_splitter = TokenTextSplitter(chunk_size=MAX_CHUNK_TOKEN_SIZE, chunk_overlap=int(MAX_CHUNK_TOKEN_SIZE*CHUNK_OVERLAPPING))
    
    chunks = text_splitter.split_text(text)
    for j,chunk in enumerate(chunks):
        new_filename = filename[:-3]+"_"+str(j)+".txt"
        with open(chunk_dir+"/"+new_filename, "w", encoding="utf-8") as file:
            file.write(chunk)

In [41]:
! pip install azure-search-documents

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField, SearchFieldDataType, SearchableField, SearchField, VectorSearch, HnswAlgorithmConfiguration,
    VectorSearchProfile, SemanticConfiguration, SemanticPrioritizedFields, SemanticField, SemanticSearch,
    SearchIndex, VectorSearchAlgorithmKind, HnswParameters, VectorSearchAlgorithmMetric
)
##CREATE INDEX##
#Create search endpoint
ai_search_endpoint = os.getenv("AI_SEARCH_ENDPOINT")
ai_search_apikey = os.getenv("AI_SEARCH_KEY")
ai_search_index_name = os.getenv("AI_SEARCH_INDEX_NAME")
ai_search_credential = AzureKeyCredential(ai_search_apikey)
index_client = SearchIndexClient(endpoint=ai_search_endpoint, credential=ai_search_credential)

dimensions = 1536
# Fields definition
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(name="embeddingTitle", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=dimensions, vector_search_profile_name="myHnswProfile"),
    SearchField(name="embeddingContent", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=dimensions, vector_search_profile_name="myHnswProfile")
]

# Configure the vector search configuration
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        )
    ]
)

# Semantic ranker configuration
semantic_config = SemanticConfiguration(
    name="semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=ai_search_index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)


In [ ]:
from azure.search.documents import SearchIndexingBufferedSender
from openai import AzureOpenAI
import time
##FILL INDEX WITH CHUNKS##
#Create embedding endpoint
endpoint = os.getenv("FOUNDRY_ENDPOINT")
subscription_key = os.getenv("FOUNDRY_KEY")
embedding_model_name = os.getenv("EMBEDDING_MODEL_NAME")
api_version = "2024-12-01-preview"
embedding_client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key
)


batch_client = SearchIndexingBufferedSender(
                endpoint=ai_search_endpoint,
                index_name=ai_search_index_name,
                credential=ai_search_credential
            )
chunk_files = os.listdir(chunk_dir)
lote = []
for i, chunk_name in  enumerate(chunk_files):
    with open(chunk_dir+"/"+chunk_name, "r", encoding="utf-8") as f:
        content = f.read()
    title = chunk_name[:-4]
    document = {
        "id": str(i),
        "title": title,
        "content": content,
        "embeddingTitle": embedding_client.embeddings.create(input=title, model=embedding_model_name).data[0].embedding,
        "embeddingContent": embedding_client.embeddings.create(input=content, model=embedding_model_name).data[0].embedding,
    }
    lote.append(document)
    if (i + 1) % 32 == 0:
        # Upload documents
        print(f'INDEXING BATCH {i + 1}')
        try:
            batch_client.upload_documents(documents=lote)
            print('Waiting 15 seconds...')
            time.sleep(15)
        except Exception as ex:
            print(ex)
        lote = []
if len(lote)>0:
    try:
        batch_client.upload_documents(documents=lote)
        print('Waiting 15 seconds...')
        time.sleep(15)
    except Exception as ex:
        print(ex)


ERROR! Session/line number was not unique in database. History logging moved to new session 72
INDEXING BATCH 32
Waiting 15 seconds...
INDEXING BATCH 64
Waiting 15 seconds...
INDEXING BATCH 96
Waiting 15 seconds...
INDEXING BATCH 128
Waiting 15 seconds...
INDEXING BATCH 160
Waiting 15 seconds...
Waiting 15 seconds...
